In [ ]:
import pandas as pd
import numpy as np
import re

In [ ]:
data=pd.read_excel('赛题1数据材料\赛题1数据集\上市信息财务信息资产负债表.xlsx')

In [ ]:
data

In [ ]:
#将数字后的单位全部去除,只匹配数字，其余置为nan。
def convert(cell):
    pattern=r'[-\d.]'
    if cell == '--' or len(re.findall(pattern,cell)) == 0:
        return np.nan
    return float(''.join(re.findall(pattern, cell)))

In [ ]:
datanew=data.iloc[:,3:]

In [ ]:
datanew.applymap(str).applymap(convert)

In [ ]:
#将两个表链接起来
data=data.iloc[:,:3].join(datanew)

In [ ]:
data

In [ ]:
data['日期'].value_counts()

In [ ]:
#补全时间，规整时间的格式
pattern='\d\d\d\d-\d\d-\d\d'
for i in range(len(data)):
    if re.findall(pattern,data.iloc[i,2])==[]:
        data.iloc[i,2]='20'+data.iloc[i,2]

In [ ]:
#去除企业编号与日期值同时相同的行，保留第一次出现的行
data.drop_duplicates(subset=['企业编号','日期'],keep='first',inplace=True)

In [ ]:
#再次查看出现较多的时间
data['日期'].value_counts()

In [ ]:
#将数量较少的日期全部丢弃
timelist=list(['2017-09-30','2017-12-31','2018-06-30','2018-09-30','2017-06-30','2018-03-31'])
data=data[data['日期'].isin(timelist)]
data

In [ ]:
#转换数据格式
def get_number(cell):
    if not isinstance(cell, str):
        return cell
    _ = re.findall(r'[\d.]+',cell)
    if not _:
        return None
    return float(_[0])

In [ ]:
data.iloc[:,3:]=data.iloc[:,3:].applymap(get_number)

In [ ]:
data.sort_values(by=['企业编号','日期'],inplace=True)

In [ ]:
#查看排序结果是否达到目标效果
data.columns

In [ ]:
def fuckdirty(cell):
    if cell>100:
        return None
    else:
        return cell

In [ ]:
data.iloc[:,5:23]=data.iloc[:,5:23].applymap(fuckdirty)

In [ ]:
data.columns=pd.Index(['企业编号', '标题', '日期', '$资产:货币资金(元)', '$资产:应收账款(元)', '$资产:其它应收款(元)',
       '$资产:存货(元)', '$资产:流动资产合计(元)', '$资产:长期股权投资(元)', '$资产:累计折旧(元)', '$资产:固定资产(元)',
       '$资产:无形资产(元)', '$资产:资产总计(元)', '$负债:应付账款(元)', '$负债:预收账款(元)', '$负债:存货跌价准备(元)',
       '$负债:流动负债合计(元)', '$负债:长期负债合计(元)', '$负债:负债合计(元)', '$权益:实收资本(或股本)(元)',
       '$权益:资本公积金(元)', '$权益:盈余公积金(元)', '$权益:股东权益合计(元)', '流动比率'])

In [ ]:
# '资产:货币资金(元)',资产:应收账款(元),
# data['资产:长期股权投资(元)'].value_counts()
# data['资产:长期股权投资(元)'].describe()

In [ ]:
data=data.set_index(['企业编号','日期'])
data=data.unstack().reset_index().set_index('企业编号').T.drop('标题').drop('流动比率')

In [ ]:
data=data.reindex(columns=list(range(1001,4001)))
data.to_pickle('上市信息财务信息资产负债表.pickle')
data

In [ ]:
data.index